In [53]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense ,GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [35]:
df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [36]:
df['sentiment']=df['sentiment'].replace({'positive':1,'negative':0})
df.head()

C:\Users\MEGHANSH\AppData\Local\Temp\ipykernel_28528\1966035809.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment']=df['sentiment'].replace({'positive':1,'negative':0})


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [37]:
import re
import string
from bs4 import BeautifulSoup

def clean_text(text):
    # Remove HTML tags using BeautifulSoup
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove special keywords (if any specific set of keywords are defined, use them)
    # Example of removing keywords like 'SPECIAL_KEYWORD'
    keywords_to_remove = ['SPECIAL_KEYWORD1', 'SPECIAL_KEYWORD2']
    for keyword in keywords_to_remove:
        text = re.sub(r'\b' + re.escape(keyword) + r'\b', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove special characters and extra spaces
    text = re.sub(r'[^A-Za-z0-9\s]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [38]:
df['review']=df['review'].apply(clean_text)

C:\Users\MEGHANSH\AppData\Local\Temp\ipykernel_28528\290461631.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [39]:
df['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production The filming tech...
2        I thought this was a wonderful way to spend ti...
3        Basically theres a family where a little boy J...
4        Petter Matteis Love in the Time of Money is a ...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [40]:
# split data into training data and test data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [41]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [42]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=300)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=300)

In [43]:
print(X_train)

[[  22   39  919 ...  205  335 3771]
 [   0    0    0 ...   88  102    8]
 [   0    0    0 ...    2  721   64]
 ...
 [   0    0    0 ... 1601    2  627]
 [   0    0    0 ...  241  102  121]
 [   0    0    0 ...   65   71 1938]]


In [44]:
print(X_test)

[[   0    0    0 ...    8  969 2950]
 [  13  103  267 ...   63   46    9]
 [   0    0    0 ...   50 1056   89]
 ...
 [   0    0    0 ...  121  195 3320]
 [   0    0    0 ... 1043    1 2208]
 [   0    0    0 ...    1  334   27]]


In [45]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [46]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [47]:
Y_test

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64

In [73]:
# build the model

model_2 = Sequential()
model_2.add(Embedding(input_dim=5000, output_dim=128, input_length=300))
model_2.add(GRU(256, dropout=0.2,return_sequences=True))
model_2.add(GRU(256, dropout=0.2))
model_2.add(Dense(1, activation="sigmoid"))

f:\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [74]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (32, 300, 128)         │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (32, 300, 256)         │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (32, 256)              │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (32, 1)                │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,994,373 (15.24 MB)

 Trainable params: 1,331,457 (5.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,662,916 (10.16 MB)

In [75]:
# compile the model
model_2.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [78]:
model_2.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1458s 3s/step - accuracy: 0.7177 - loss: 0.5513 - val_accuracy: 0.8443 - val_loss: 0.3492
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 2816s 6s/step - accuracy: 0.8869 - loss: 0.2772 - val_accuracy: 0.8924 - val_loss: 0.2594
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 627s 1s/step - accuracy: 0.9292 - loss: 0.1836 - val_accuracy: 0.8931 - val_loss: 0.2717
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.9445 - loss: 0.1507 - val_accuracy: 0.8915 - val_loss: 0.2944
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.9579 - loss: 0.1160 - val_accuracy: 0.8857 - val_loss: 0.3134


In [79]:
loss, accuracy = model_2.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 55s 174ms/step - accuracy: 0.8896 - loss: 0.2921
Test Loss: 0.28994762897491455
Test Accuracy: 0.8912000060081482


In [72]:
import pickle

# Assuming 'tokenizer' was used for training
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [80]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model_2.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [82]:
# example usage
new_review = "This movie is bad"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
The sentiment of the review is: negative


In [87]:
# example usage
new_review = "Fantastic"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
The sentiment of the review is: positive


In [83]:
model_2.save('sentiment_model.h5')

In [84]:
TF_ENABLE_ONEDNN_OPTS=0